In [ ]:
# --- PyTorch ---
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset # <-- Usaremos o DataLoader padrão

# --- Hugging Face Transformers (BERT) ---
from transformers import BertTokenizer, BertModel, logging
# Desliga os warnings do Hugging Face (opcional, mas limpa a saída)
logging.set_verbosity_error() 

# --- PyTorch Geometric (GNNs) ---
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GCNConv, global_mean_pool

# --- NLTK (Stopwords) ---
import nltk
from nltk.corpus import stopwords

# --- Scikit-learn (Métricas) ---
from sklearn.metrics import accuracy_score

# --- Verificação de Hardware (CRÍTICO) ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Usando dispositivo: {device}")
if torch.cuda.is_available():
    print(f"Nome da GPU: {torch.cuda.get_device_name(0)}")
print("---------------------------------")

In [ ]:
# --- Baixar Stopwords do NLTK ---
nltk.download('stopwords')
stop_words_set = set(stopwords.words('english'))
print(f"Carregadas {len(stop_words_set)} stopwords do NLTK.")

# --- Constantes do Projeto ---
BERT_MODEL_NAME = 'bert-base-uncased'
DATA_DIR = "./data" # !!! MUDE ESTE CAMINHO !!!

# --- Hiperparâmetros de Treinamento ---
# 2e-5 é o learning rate padrão para fine-tuning de BERT
LEARNING_RATE = 2e-5 
BATCH_SIZE = 2 # Ajuste se tiver erros de memória (pode tentar 8 ou 12)
NUM_EPOCHS = 4  # Fine-tuning converge rápido. Comece com 3-5
HIDDEN_DIM = 256 # Dimensão das camadas da GNN
MAX_LENGTH = 200 # Limite máximo de tokens do BERT

### Para Carregar Dataset IMDB

In [ ]:
from datasets import load_dataset

# --- 1. Carregar Dataset IMDB ---
print("Baixando e carregando dataset IMDB...")
# Isso baixa o dataset (pode demorar um pouco na primeira vez)
imdb_dataset = load_dataset("imdb")

print("Formatando dados...")
# O IMDB já vem com 'text' e 'label' (0=neg, 1=pos)
# E já vem pré-dividido em 'train' e 'test'
train_texts_cleaned = imdb_dataset['train']['text']
train_labels = imdb_dataset['train']['label']

test_texts_cleaned = imdb_dataset['test']['text']
test_labels = imdb_dataset['test']['label']

# --- 2. Definir Classes (Hardcoded) ---
num_classes = 8 # IMDB é binário (negativo/positivo)

print(f"--- Dados do IMDB Carregados ---")
print(f"Total de classes: {num_classes}")
print(f"Documentos de treino: {len(train_texts_cleaned)}")
print(f"Documentos de teste: {len(test_texts_cleaned)}")

# --- 3. Criar a classe TextGraphDataset ---
# (Esta classe é da sua Célula 4 original, apenas a copiamos para cá)
class TextGraphDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# --- 4. Criar instâncias do Dataset ---
train_dataset = TextGraphDataset(train_texts_cleaned, train_labels)
test_dataset = TextGraphDataset(test_texts_cleaned, test_labels)

# --- 5. Carregar Tokenizador (da Célula 4 original) ---
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

def build_token_graph(num_tokens, window_size=2):
    """Cria um grafo de janela deslizante para 'num_tokens'."""
    edge_index_list = []
    for i in range(num_tokens):
        for j in range(max(0, i - window_size), min(num_tokens, i + window_size)):
            if i != j:
                edge_index_list.append([i, j])
                
    if not edge_index_list:
        return torch.tensor([[0], [0]], dtype=torch.long)
    
    return torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()

### Para Utilizar Dataset R8

In [ ]:
def load_data_and_labels(filepath):
    """Lê os arquivos de dados do R8 (formato: label\ttexto)."""
    texts = []
    labels = []
    with open(filepath, 'r', encoding='latin-1') as f:
        for line in f:
            line = line.strip()
            if not line: continue
            parts = line.split('\t', 1)
            if len(parts) == 2:
                label, text = parts
                texts.append(text)
                labels.append(label)
    return texts, labels

def clean_text(text):
    """Limpa o texto (remove pontuação, números, stopwords)."""
    text = re.sub(r"[^a-zA-Z]", " ", text) # Mantém apenas letras
    text = text.lower()
    words = text.split()
    cleaned_words = [w for w in words if w not in stop_words_set and len(w) > 2]
    return " ".join(cleaned_words) # Retorna o texto limpo como string

# --- Carregar Dados ---
print("Carregando e limpando dados...")
train_texts_raw, train_labels_str = load_data_and_labels(os.path.join(DATA_DIR, 'train.txt'))
test_texts_raw, test_labels_str = load_data_and_labels(os.path.join(DATA_DIR, 'test.txt'))

train_texts_cleaned = [clean_text(text) for text in train_texts_raw]
test_texts_cleaned = [clean_text(text) for text in test_texts_raw]

# --- Mapeamento de Labels ---
labels_unique = sorted(list(set(train_labels_str)))
num_classes = len(labels_unique)
label_map = {label: i for i, label in enumerate(labels_unique)}

train_labels = [label_map[label] for label in train_labels_str]
test_labels = [label_map[label] for label in test_labels_str]

print(f"Total de classes: {num_classes} ({labels_unique[0]}, {labels_unique[1]}, ...)")
print(f"Documentos de treino: {len(train_texts_cleaned)}")
print(f"Documentos de teste: {len(test_texts_cleaned)}")

In [ ]:
# Carrega o tokenizador globalmente
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME)

def build_token_graph(num_tokens, window_size=2):
    """Cria um grafo de janela deslizante para 'num_tokens'."""
    edge_index_list = []
    for i in range(num_tokens):
        for j in range(max(0, i - window_size), min(num_tokens, i + window_size)):
            if i != j:
                edge_index_list.append([i, j])
                
    if not edge_index_list:
        # Garante que o grafo não esteja vazio (para [CLS], [SEP])
        return torch.tensor([[0], [0]], dtype=torch.long)
    
    return torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()


class TextGraphDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Retorna o texto bruto e a label
        return self.texts[idx], self.labels[idx]

# Cria as instâncias do Dataset
train_dataset = TextGraphDataset(train_texts_cleaned, train_labels)
test_dataset = TextGraphDataset(test_texts_cleaned, test_labels)

In [ ]:
def collate_fn_bert_gnn(batch):
    """
    Função customizada para empacotar o lote.
    'batch' é uma lista de tuplas (texto, label)
    """
    texts, labels = zip(*batch)
    
    # 1. Processa para o BERT
    # Tokeniza o lote de textos com padding
    bert_inputs = tokenizer(
        list(texts),
        padding=True,
        truncation=True,
        max_length=MAX_LENGTH,
        return_tensors="pt"
    )
    
    # 2. Processa para a GNN
    data_list = []
    for i in range(len(texts)):
        # Pega o número real de tokens (sem padding)
        # O 'attention_mask' é 1 para tokens reais, 0 para padding
        num_tokens = int(bert_inputs['attention_mask'][i].sum())
        
        # Constrói o grafo de tokens para este item
        edge_index = build_token_graph(num_tokens)
        
        # O 'y' (label) para este item
        y = torch.tensor(labels[i], dtype=torch.long)
        
        # Criamos um Data object sem 'x'. O 'x' virá do BERT
        data_list.append(Data(edge_index=edge_index, y=y))
    
    # 3. Empacota a GNN
    # 'Batch.from_data_list' cria o batch de grafos para o PyG
    pyg_batch = Batch.from_data_list(data_list)
    
    # 4. Retorna tudo
    return {
        'bert_inputs': bert_inputs, # Contém input_ids, attention_mask
        'pyg_batch': pyg_batch        # Contém edge_index, y, batch_map
    }

In [ ]:
# Usa o DataLoader padrão do PyTorch, mas com nosso collate_fn
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_fn_bert_gnn,
    num_workers=0,
    pin_memory=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_fn_bert_gnn,
    num_workers=0,
    pin_memory=False
)

print("--- DataLoaders Prontos ---")
print(f"Total de lotes de treino: {len(train_loader)}")
print(f"Total de lotes de teste: {len(test_loader)}")

In [ ]:
class HybridGNN_FineTune(nn.Module):
    def __init__(self, hidden_dim, output_dim, dropout_prob=0.5):
        super(HybridGNN_FineTune, self).__init__()
        
        # 1. Camada BERT (Será treinada)
        self.bert = BertModel.from_pretrained(BERT_MODEL_NAME)
        bert_output_dim = self.bert.config.hidden_size # 768
        
        # 2. Camadas GNN
        self.gcn1 = GCNConv(bert_output_dim, hidden_dim)
        self.gcn2 = GCNConv(hidden_dim, hidden_dim)
        
        # 3. Pooling (Agrega os nós de tokens em um vetor de documento)
        self.pool = global_mean_pool
        
        # 4. Camada de Classificação Final
        self.classifier = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(p=dropout_prob)

    def forward(self, bert_inputs, pyg_batch):
        # Move os inputs do BERT para a GPU
        input_ids = bert_inputs['input_ids'].to(device)
        attention_mask = bert_inputs['attention_mask'].to(device)
        
        # Move os inputs da GNN para a GPU
        edge_index = pyg_batch.edge_index.to(device)
        batch_map = pyg_batch.batch.to(device) # Mapa de nós para grafos
        
        # 1. Passa pelo BERT (Fine-tuning)
        # Pega os embeddings de CADA token
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        token_embeddings = outputs.last_hidden_state # Shape: [batch_size, seq_len, 768]

        # 2. Prepara Features para a GNN
        # A GNN espera um 'x' (features) no formato [Total de Nós, Dim Features]
        # Precisamos "desenrolar" os embeddings do BERT
        
        # Pega apenas os embeddings dos tokens reais (ignora padding)
        # O 'batch_map' do PyG nos diz quais tokens pertencem a qual doc
        # Esta é uma forma eficiente de fazer isso:
        active_token_embeddings = token_embeddings[attention_mask > 0]
        
        # 'active_token_embeddings' é agora o nosso 'x' para a GNN
        
        # 3. Passa pela GNN
        h = self.gcn1(active_token_embeddings, edge_index)
        h = F.relu(h)
        h = self.gcn2(h, edge_index)
        h = F.relu(h)
        
        # 4. Pooling (Agrega os nós em um vetor por grafo)
        # h: (Total de Nós no Batch, hidden_dim)
        # graph_embedding: (Tamanho do Batch, hidden_dim)
        graph_embedding = self.pool(h, batch_map)
        
        # 5. Classificação
        graph_embedding = self.dropout(graph_embedding)
        logits = self.classifier(graph_embedding)
        
        return logits

# --- Instanciando o Modelo ---
model = HybridGNN_FineTune(
    hidden_dim=HIDDEN_DIM,
    output_dim=num_classes, # 8
    dropout_prob=0.5
).to(device)

criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("--- Arquitetura do Modelo (com Fine-Tuning) ---")
# print(model) # Descomente se quiser ver a arquitetura completa
print(f"Modelo {type(model).__name__} instanciado e movido para {device}.")

In [ ]:
print(f"--- Iniciando Treinamento por {NUM_EPOCHS} épocas ---")
print(f"Otimizador: Adam, LR: {LEARNING_RATE}, Batch Size: {BATCH_SIZE}")

for epoch in range(NUM_EPOCHS):
    
    model.train() # Coloca o modelo em modo de treino
    total_loss = 0
    start_time = time.time()
    
    for i, batch in enumerate(train_loader):
        # O batch é um dicionário {'bert_inputs': ..., 'pyg_batch': ...}
        bert_inputs = batch['bert_inputs']
        pyg_batch = batch['pyg_batch']
        
        # Pega as labels corretas
        labels = pyg_batch.y.to(device)
        
        # Zera os gradientes
        optimizer.zero_grad()
        
        # Forward pass
        logits = model(bert_inputs, pyg_batch)
        
        # Calcula o loss
        loss = criterion(logits, labels)
        
        # Backward pass (Calcula os gradientes para GNN e BERT)
        loss.backward()
        
        # Otimiza (Ajusta os pesos)
        optimizer.step()
        
        total_loss += loss.item()
        
        if (i + 1) % 50 == 0:
            print(f"  Época {epoch+1}, Lote {i+1}/{len(train_loader)} | Loss Lote: {loss.item():.4f}")

    # --- Fim da Época ---
    end_time = time.time()
    avg_loss = total_loss / len(train_loader)
    epoch_time = end_time - start_time
    
    print(f"\nÉPOCA {epoch+1:02d}/{NUM_EPOCHS} Completa")
    print(f"  Tempo: {epoch_time:.2f}s")
    print(f"  Loss Média (Treino): {avg_loss:.4f}")

print("--- Treinamento Concluído ---")

In [ ]:
print("--- Iniciando Avaliação no Conjunto de Teste (Indutivo) ---")

model.eval() # Coloca o modelo em modo de avaliação
all_predictions = []
all_true_labels = []

# Loop de avaliação com limpeza de cache
with torch.no_grad(): # Desliga o cálculo de gradientes
    
    # Adiciona um timer para ver a velocidade
    start_time = time.time() 
    
    for i, batch in enumerate(test_loader):
        bert_inputs = batch['bert_inputs']
        pyg_batch = batch['pyg_batch']
        
        labels = pyg_batch.y.to(device)
        
        # Faz a previsão
        logits = model(bert_inputs, pyg_batch)
        
        # Pega a classe com maior pontuação
        _, predicted = torch.max(logits, dim=1)
        
        # Salva para cálculo final
        all_predictions.extend(predicted.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

        # --- [NOVO] LIMPEZA MANUAL DE MEMÓRIA ---
        # Deleta as referências aos tensores da GPU
        del bert_inputs, pyg_batch, labels, logits, predicted
        
        # Força o PyTorch a esvaziar o cache da VRAM
        torch.cuda.empty_cache()
        # ---------------------------------------

# --- Calcula a Acurácia Final ---
end_time = time.time()
accuracy = accuracy_score(all_true_labels, all_predictions)

print(f"\nTempo total de avaliação: {(end_time - start_time):.2f} segundos")
print("\n--- Resultados Finais (com Fine-Tuning) ---")
print(f"Total de Amostras de Teste: {len(all_true_labels)}")
print(f"Acurácia no Teste: {accuracy * 100:.2f}%")

In [ ]:
# --- 1. Definir o caminho para salvar ---
dataset = "imdb"  # Altere conforme o dataset utilizado
MODEL_SAVE_PATH = f"{dataset}_hybrid_gnn_finetuned.pth"

# 2. Salvar o 'state_dict' (os pesos)
# Isso salva apenas os parâmetros aprendidos, não a arquitetura
torch.save(model.state_dict(), MODEL_SAVE_PATH)

print(f"\n--- Modelo Salvo ---")
print(f"Os parâmetros do modelo foram salvos em: {MODEL_SAVE_PATH}")

### Apenas para carregamento

In [ ]:
# --- 1. Definir o caminho do modelo salvo ---
MODEL_SAVE_PATH = "r8_hybrid_gnn_finetuned.pth"
num_classes = 8

# --- 2. Recriar a arquitetura do modelo ---
# Você DEVE instanciar a classe do modelo primeiro.
# (Certifique-se de que HIDDEN_DIM e num_classes estão definidos pela Célula 2)
print("Instanciando a arquitetura do modelo...")
loaded_model = HybridGNN_FineTune(
    hidden_dim=HIDDEN_DIM,
    output_dim=num_classes,
).to(device)

# --- 3. Carregar os pesos salvos ---
print(f"Carregando pesos de {MODEL_SAVE_PATH}...")
loaded_model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))

# --- 4. Colocar em modo de avaliação (MUITO IMPORTANTE!) ---
# Isso desliga o dropout e outras camadas específicas de treino.
loaded_model.eval()

print("\n--- Modelo Carregado e Pronto! ---")
print("O 'loaded_model' está pronto para ser usado para avaliação (Célula 9).")